In [20]:
# Import libraries
import pandas as pd
import numpy as np
import math

import geopandas as gpd
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

In [21]:
def style(p):
    # Title 
    p.title.align = 'center'
    p.title.text_font_size = '20pt'
    p.title.text_font = 'serif'

    # Axis titles
    p.xaxis.axis_label_text_font_size = '14pt'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = '14pt'
    p.yaxis.axis_label_text_font_style = 'bold'

    # Tick labels
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'

    return p

In [22]:
# Import libraries
import pandas as pd
import numpy as np
import math

import geopandas as gpd
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter, Text
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.layouts import widgetbox, row, column
from bokeh.models import (CategoricalColorMapper, HoverTool, 
                          ColumnDataSource, Panel, 
                          FuncTickFormatter, SingleIntervalTicker, LinearAxis)
from bokeh.models.widgets import (CheckboxGroup, Slider, RangeSlider, 
                                  Tabs, CheckboxButtonGroup, 
                                  TableColumn, DataTable, Select)


In [42]:
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

In [62]:
def map_tab(doc):

    df = pd.read_csv('../data/DC_Properties.csv')   
    df.loc[:,'SALEDATE'] = pd.to_datetime(df.loc[:,'SALEDATE'],format='%Y-%m-%d %H:%M:%S')
    df.loc[:,'SALEYEAR'] = [x.year for x in df.SALEDATE]

    # Fill house square foot zero values with the average house square footage by bedroom
    average_data = df.groupby('BEDRM').LANDAREA.mean()
    # Use average landarea by bedroom for each 0 value in each bedroom group, up to 14 bedrooms 
    for i in range(0, 14): 
        df.loc[(df['LANDAREA'] == 0) & (df['BEDRM'] == i), 'LANDAREA'] = average_data.loc[i]

    df['price_sf'] = df['PRICE'] / df['LANDAREA']
    df = df[df['SALEYEAR'].notna()]
    df = df[df['ASSESSMENT_NBHD'].notna()]
    df = df[df['PRICE'].notna()]
    df = df[df['LANDAREA'].notna()]
    df = df[df['price_sf'].notna()]
    df = df[df['SALEDATE'].notna()]
    neighborhood_data = df.groupby(
        ['SALEYEAR', 'ASSESSMENT_NBHD', 'CENSUS_TRACT']
    ).agg(
        {
        'PRICE': ['count', 'mean', 'median'],
        'LANDAREA': ['mean'],
        'price_sf': ['mean']
        }
    )

    #Reset the index to 1 level to fill in year
    neighborhood_data = neighborhood_data.set_axis(neighborhood_data.columns.map('_'.join), axis=1, inplace=False)
    neighborhood_data = neighborhood_data.reset_index(level=[0,1])

    # Change data types to integer for price_sf and year
    neighborhood_data = neighborhood_data.astype({'PRICE_mean': 'int'})
    neighborhood_data = neighborhood_data.astype({'PRICE_median': 'int'})
    neighborhood_data = neighborhood_data.astype({'LANDAREA_mean': 'int'})
    # neighborhood_data = neighborhood_data.astype({'price_sf_mean': 'int'})
    neighborhood_data = neighborhood_data.astype({'SALEYEAR': 'int'})

    # Read in shapefile and examine data
    dc = gpd.read_file('../data/Census_Tracts_in_2010.shp')
    # Set the Coordinate Referance System (crs) for projections
    # ESPG code 4326 is also referred to as WGS84 lat-long projection
    dc.crs = {'init': 'epsg:4326'}

    # Rename columns in geojson map file
    dc = dc.rename(columns={'geometry': 'geometry'}).set_geometry('geometry')


    dc.sort_values(by=['TRACT'])

    neighborhood_data.index = neighborhood_data.index.astype(int)
    dc.TRACT = dc.TRACT.astype(int)

    # This dictionary contains the formatting for the data in the plots
    format_data = [('PRICE_count', 0, 100,'0,0', 'Number of properties remodeled'),
                ('PRICE_mean', 0, 1_500_000,'$0,0', 'Average Sales Price'),
                ('PRICE_median', 0, 1_500_000, '$0,0', 'Median Sales Price'),
                ('LANDAREA_mean', 500, 5000,'0,0', 'Average Square Footage'),
                ('price_sf_mean', 0, 2000,'$0,0', 'Average Price Per Square Foot')]
    
    #Create a DataFrame object from the dictionary 
    format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])

    def json_data(selectedYear):
        yr = selectedYear

        # Pull selected year from neighborhood summary data
        df_yr = neighborhood_data[neighborhood_data['SALEYEAR'] == yr]

        # Merge the GeoDataframe object (dc) with the neighborhood summary data (neighborhood)
        merged = pd.merge(dc, df_yr, left_on='TRACT', right_index=True, how='left')

        # Fill the null values
        values = {'year': yr, 'PRICE_count': 0, 'PRICE_mean': 0, 'PRICE_median': 0,
                    'sf_mean': 0, 'price_sf_mean': 0, 'ASSESSMENT_NBHD': "", 'LANDAREA_mean': 0}
        merged = merged.fillna(value=values)

        # Bokeh uses geojson formatting, representing geographical features, with json
        # Convert to json
        merged_json = json.loads(merged.to_json())

        # Convert to json preferred string-like object 
        json_data = json.dumps(merged_json)
        return json_data
    
    def style(p):
        # Title 
        p.title.align = 'center'
        p.title.text_font_size = '20pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

        return p
    
    # Create a plotting function
    def make_plot(field_name):    
        # Set the format of the colorbar
        min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
        max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
        field_format = format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]

        # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
        color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)

        # Create color bar.
        format_tick = NumeralTickFormatter(format=field_format)
        color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, formatter=format_tick,
        border_line_color=None, location = (0, 0))

        # Create figure object.
        verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]

        p = figure(title = verbage + ' by Neighborhood for Homes in DC by Year remodeled', 
                    plot_height = 700, plot_width = 1000,
                    toolbar_location = None)
        p.xgrid.grid_line_color = None
        p.ygrid.grid_line_color = None
        p.axis.visible = False

        # Add patch renderer to figure. 
        p.patches('xs','ys', source = geosource, fill_color = {'field' : field_name, 'transform' : color_mapper},
                line_color = 'black', line_width = 0.25, fill_alpha = 1)

        # Specify color bar layout.
        p.add_layout(color_bar, 'right')

        # Add the hover tool to the graph
        p.add_tools(hover)
        return p
    
    # Define the callback function: update_plot
    def update_plot(attr, old, new):
        # The input yr is the year selected from the slider
        yr = slider.value
        new_data = json_data(yr)

        # The input cr is the criteria selected from the select box
        cr = select.value
        input_field = format_df.loc[format_df['verbage'] == cr, 'field'].iloc[0]

        # Update the plot based on the changed inputs
        p = make_plot(input_field)

        # Update the layout, clear the old document and display the new document
        #layout = column(p, widgetbox(select), widgetbox(slider))
        #curdoc().clear()
        #curdoc().add_root(layout)

        # Update the data
        geosource.geojson = new_data
        
    # Input geojson source that contains features for plotting 
    geosource = GeoJSONDataSource(geojson = json_data(2002))
    input_field = 'PRICE_median'

    # Define a sequential multi-hue color palette.
    palette = inferno(100)

    # Reverse color order so that dark blue is highest obesity.
    #palette = palette[::-1]

        # Add hover tool
    hover = HoverTool(tooltips = [ ('Neighbourhood','@ASSESSMENT_NBHD'),
                                    ('Number of properties remodeled', '@PRICE_count'),
                                    ('Average Price', '$@PRICE_mean{,}'),
                                    ('Median Price', '$@PRICE_median{,}'),
                                    ('Average landarea', '@LANDAREA_mean{,}'),
                                    ('Price/SF ', '$@price_sf_mean{,}')])

    # Call the plotting function
    p = make_plot(input_field)
    p = style(p)
    # Make a slider object: slider 
    slider = Slider(title = 'Year',start = 1990, end = 2018, step = 1, value = 2000)
    #slider.callback_policy = "mouseup"
    slider.on_change('value_throttled', update_plot)

    # Make a selection object: select
    select = Select(title='Select Criteria:', value='Median Sales Price', options=['Median Sales Price', 'Average Sales Price',
                                                                                    'Average Price Per Square Foot',
                                                                                    'Average Square Footage', 'Number of Sales'])
    select.on_change('value', update_plot)

    # Make a column layout of widgetbox(slider) and plot, and add it to the current document
    # Display the current document
    layout = column(p, widgetbox(select), widgetbox(slider))

    # Make a tab with the layout 
    tab = Panel(child=layout, title = 'Map')
    tabs = Tabs(tabs=[tab])
    
    doc.add_root(tabs)
    
# Set up an application
handler = FunctionHandler(map_tab)
app = Application(handler)

In [64]:
show(app, 'localhost:8888')

RuntimeError: no display hook installed for notebook type None